In [76]:
################################
# Solves du/dt = u''           #
# With zero Dirichlet boundary #
################################
import radiant as rad
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display_html


# Problem Parameters
a = 0.
b = 1.
alpha = 1.


def f(t, *x):
    return np.zeros_like(x[0])


def g(t, *x):
    return np.zeros_like(x[0])


def u0(*x):
    return np.prod(np.sin(np.pi * np.asarray(x)), axis=0)


def exact(t, *x):
    return np.exp(- np.pi ** 2 * alpha * t) * np.prod(np.sin(np.pi * np.asarray(x)), axis=0)

In [77]:
# Parameters
d = 1
k = 2
tdelta = 0.2
xdelta = 0.1
tN = 100
xN = 100

# Computed Parameters
delta = np.array([tdelta] + [xdelta] * d)
xranges = ((a, b),) * d
ranges = ((0., 1.), *xranges)
xc = rad.gridn(ranges, (tN, *((xN,) * d)), flat=True)
Lidx = lambda cs: np.all([np.logical_and(c != a, c != b) for c in cs], axis=0)
Bidx = lambda cs: np.any([np.logical_or(c == a, c == b) for c in cs], axis=0)

L = lambda func: lambda *x: - alpha * np.sum([func(*x, m=(i, i)) for i in range(1, d+1)], axis=0)
B = lambda func: lambda *x: func(*x)

# Solve for approximate solution
phi = rad.Wendland(d+1, k, delta, xc)
solver = rad.solve.SpaceTimeCollocation(phi, L, Lidx, B, Bidx)
approx = solver.solve(f, g, u0)

In [78]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)
ax.margins(x=0.)

xs = np.linspace(a, b, 100)
exact_line, = ax.plot(xs, exact(0, xs), label="Exact")
approx_line, = ax.plot(xs, approx(*np.meshgrid(0, xs)), label="Approximate")
plt.legend()

ts = np.linspace(0, 1, 100)
def func(i):
    tx = np.meshgrid(ts[i], xs)
    approx_line.set_ydata(approx(*tx))
    exact_line.set_ydata(exact(ts[i], xs))
    return approx_line, exact_line,


anim = animation.FuncAnimation(
    fig,
    func,
    frames=len(ts),
    interval=200,
    blit=True,
    repeat=True
)

display_html(HTML(anim.to_jshtml()))
anim.save('figures/st_heat.gif', writer=animation.FFMpegWriter(fps=30)) 
plt.close()

Once 
 
 Loop 
 
 Reflect